<a href="https://colab.research.google.com/github/juananthony/gans-networks/blob/master/workshop_exercise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paso 1 - Instalando paquetes en Notebooks



Paso 2 - Iniciando TensorBoard

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Paso 3 - Importando Paquetes

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time
import datetime

from PIL import Image
from IPython import display
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, LeakyReLU, Reshape, Conv2DTranspose, Conv2D, Dropout, Flatten
from keras import optimizers

Paso 4 - Definición de variable globales

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
WIDTH = 28
HEIGHT = 28

Paso 5 - Carga de los datos

In [ ]:
def generate_data_set(height, weight, normalization_value):
  (train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
  train_images = train_images.reshape(train_images.shape[0], height, weight, 1).astype('float32')
  train_images = (train_images - normalization_value) / normalization_value
  return train_images, train_labels

Paso 6: Creación del modelo generador (Generator)

In [ ]:
def build_generator_model(name, height, width, dims=1):
    
  model = Sequential()
  model.add(Dense(7*7*256, use_bias=False, input_shape=(height*width, )))
  model.add(BatchNormalization())
  model.add(LeakyReLU())

  model.add(Reshape((7, 7, 256)))

  model.add(Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
  model.add(BatchNormalization())
  model.add(LeakyReLU())

  model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
  model.add(BatchNormalization())
  model.add(LeakyReLU())

  model.add(Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

  return model

Paso 7: Creación del modelo discriminador (Discriminator)

In [ ]:
def build_discriminator_model(name, height, width, dims):
    
  model = Sequential()
  
  model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[height, width, dims]))
  model.add(LeakyReLU())
  model.add(Dropout(0.3))

  model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
  model.add(LeakyReLU())
  model.add(Dropout(0.3))

  model.add(Flatten())
  model.add(Dense(1))

  return model

Paso 8 - Definición de la clase Model

In [ ]:
class Model:

  def __init__(self, height, width, dims):
  
    self.__generator = build_generator_model('generator_test', height, width, dims)
    self.__discriminator = build_discriminator_model('discriminator_test', height, width, dims)
    self.__cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    self.__generator_optimizer = optimizers.Adam(1e-4)
    self.__discriminator_optimizer = optimizers.Adam(1e-4)

    self.__checkpoint_dir = './training_checkpoints'
    self.__checkpoint_prefix = os.path.join(self.__checkpoint_dir, "ckpt")

    self.__checkpoint = tf.train.Checkpoint(generator_optimizer=self.__generator_optimizer,
                                 discriminator_optimizer=self.__discriminator_optimizer,
                                 generator=self.__generator,
                                 discriminator=self.__discriminator)
    
    self.__checkpoint.restore(tf.train.latest_checkpoint(self.__checkpoint_dir))


  def generator_summary(self):
    return self.__generator.summary()


  def discriminator_summary(self):
    return self.__discriminator.summary() 


  @property
  def generator(self):
    return self.__generator


  @property
  def discriminator(self):
    return self.__discriminator


  @property
  def generator_optimizer(self):
    return self.__generator_optimizer
  

  @property
  def discriminator_optimizer(self):
    return self.__discriminator_optimizer

  
  @tf.autograph.experimental.do_not_convert
  def discriminator_loss(self, real_output, fake_output):
    real_loss = self.__cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = self.__cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss
  

  @tf.autograph.experimental.do_not_convert
  def generator_loss(self, fake_output):
    return self.__cross_entropy(tf.ones_like(fake_output), fake_output)

  
  def create_checkpoint(self):
    self.__checkpoint.save(file_prefix = self.__checkpoint_prefix)

Paso 9 - Definición de la función de generación de ejemplos y entrenamiento



In [ ]:
@tf.function
def train_step(model, writer, images, example_num, example_shape, epoch):

  # Definiremos un conjunto de ejemplos de tipo aleatorio
  examples = tf.random.normal([example_num, example_shape])

  with writer.as_default():
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

      generated_images = model.generator(examples, training=True)
      real_output = model.discriminator(images, training=True)
      fake_output = model.discriminator(generated_images, training=True)

      gen_loss = model.generator_loss(fake_output)
      disc_loss = model.discriminator_loss(real_output, fake_output)

    tf.summary.scalar("generator loss", gen_loss, step=epoch+1)
    tf.summary.scalar("discriminator loss", disc_loss, step=epoch+1)

    gradients_of_generator = gen_tape.gradient(gen_loss, model.generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, model.discriminator.trainable_variables)
    model.generator_optimizer.apply_gradients(zip(gradients_of_generator, model.generator.trainable_variables))
    model.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, model.discriminator.trainable_variables))


Paso 10 - Visualización y test del proceso de entrenamiento

In [ ]:
"""
Para poder comprobar como evoluciona nuestra red vamos a crear una función de test que nos permitirá analizar la evolución de nuestra red en cada iteración. Para ello crearemos una función denominada generate_and_test_images de tipo tf.function (). Esta función recibirá 5 parámetros de entrada:

model (Model) se corresponde con el modelo que está siendo entrenado.
writer (TF Writer) se corresponde con el objeto utilizado para almacena la información para evaluación el proceso de entrenamiento.
epoch (int) se corresponde con la iteración actual del proceso de entrenamiento.
num_examples (int) se corresponde con el numero de imagenes que serán generado en pasa iteración de entrenamiento.
example_shape (tupple(int, int)) se corresponde con la estructura de la imagenes.
"""
def generate_and_test_images(model, writer, epoch, num_examples, example_size):

  # Definiremos un conjunto de ejemplos de tipo aleatorio para testear nuestra RGA.
  examples = tf.random.normal([num_examples, example_size])

  # Ejecutaremos el proceso bajo el objeto writer con el objetivo de almacenar la información obtenida sobre el conjunto de test aleatorio.
  with writer.as_default():

    # Prediciremos el valor del conjunto de imagenes aleatorio generadas previamente y calcularemo el valor de los para cada una de ellas.
    predictions = model.generator(examples, training=False)
    test_loss = model.generator_loss(predictions)

    # Almacenaremos los valores de loss en el writer.
    tf.summary.scalar("test loss", test_loss, step=epoch+1)

    # generaremos una imagen con todos los ejemplos generados
    fig = plt.figure(figsize=(3,3))

    for i in range(predictions.shape[0]):
        plt.subplot(3, 3, i+1)
        plt.imshow(predictions[i, :, :, 0] * 2048 + 2048, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

Paso 11 - Definición de bucle de entrenamiento (Función)

In [ ]:
"""
Una vez que se han definido todas las variables y funciones necesarias para el proceso de aprendizaje, podemos construir el bucle en tensorflow. Esta función estará formada por 5 paramétros:

dataset (TensorFlow Slice) se corresponde con el conjunto de ejemplo de entrenamiento.
epoch (int) se corresponde con el número de iteraciones del proceso de entrenamiento.
example_num
weight (int) se corresponde con el ancho de la imagenes que se utilizarán en el proceso.
height (int) se corresponde con el ancho de la imagenes que se utilizarán en el proceso.
log_dir (str) se corresponde con la ruta donde se almacenarán los ficheros de log
"""
def train(dataset, epochs, example_num, weight, height, log_dir):

  model = Model(weight, height, 1)
  model.generator.summary()
  model.discriminator_summary()
  """
  writer = tf.summary.create_file_writer(log_dir)
  
  example_size = weight*height

  # Ejecutamos el bucle de entrenamiento en base al número de iteraciones.
  for epoch in range(epochs):
    start = time.time()

    # Para cada conjunto de imágenes presentes en el dataset realizamos el proceso de entrenamiento.
    for image_batch in dataset:
      train_step(model, writer, image_batch, example_num, example_size, epoch)

    display.clear_output(wait=True)

    # Para cada iteración aplicamos un proceso de test.
    generate_and_test_images(model, writer, epoch + 1, 9, example_size)

    # Cada 15 iteraciones almacenamos el estado del modelo con el objetivo de recuperarlo en caso de que existe 
    # algún problema durante la ejecución o en caso de que iniciomes un nuevo proceso de entrenamiento.
    if (epoch + 1) % 15 == 0:
      model.create_checkpoint()

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  display.clear_output(wait=True)

  return model
  """

log_dir = "./logs/eager" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train(None, 1, 10, WIDTH, HEIGHT, log_dir)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 12544)             9834496   
_________________________________________________________________
batch_normalization_9 (Batch (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 12544)             0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 7, 7, 128)         819200    
_________________________________________________________________
batch_normalization_10 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 7, 7, 128)        

Paso 12 - Ejecución del proceso de entrenamiento

Paso 13 - Visualización de los resultados

Paso 14: Puesta en funcionamiento de la RGA